# Sort Data 

## Imports

In [1]:
import pandas as pd
import numpy as np
import json

from geopy.distance import geodesic

## Functions

In [2]:
def census_diff(df, census):

    grp = df.groupby(["plot",  census])
    cen = grp.date.agg(['min', 'max'])
    cen["mid"] = (cen["min"] + (cen["max"] - cen["min"])/2).dt.date
    cen["difference"] = cen["mid"].diff().astype('timedelta64[D]')
    cen.loc[cen["difference"] < 0 , "difference"] = np.NAN
    cen["diff_yrs"] = cen.difference/365

    cen.reset_index(level=0, inplace=True)
    cen.reset_index(level=0, inplace=True)

    cen[census].astype(str)
    cen["step"] = cen[census].astype(str).shift() + "-" + cen[census].astype(str)
    cen.loc[cen["difference"].isnull(), "step"] = np.NaN
    cen.index = cen['plot'] + "_" + cen['step']

    return cen

## Open general data

In [3]:
# open plot locations
with open('../Data/rows.geojson') as f:
    data = json.load(f)

properties = pd.DataFrame()

# add each line of geojson file to dataframe
for i in range(0, len(data['features'])):
    properties = properties.append(pd.DataFrame(data['features'][i]['properties'], index = [i]))

# only really care about these columns
properties = properties.loc[: , ["plot_size",
                                 "centroid_y",
                                 "centroid_x",
                                 "fractal_order",
                                 "location"]]

properties.rename(columns={'centroid_y':'longitude', 'centroid_x':'latitude'}, inplace=True)
properties["longlat"] = properties.apply(lambda x: [x.longitude, x.latitude], axis=1)

# seperate dataframe for only second order fractal points
second_order = properties.loc[properties.fractal_order == 2, : ]

# function to find point closest to given point
def closest(pt, others):
    
    clst_pt = min(others.longlat, key = lambda x: geodesic(pt, x).meters)
    distnce = geodesic(pt, clst_pt).meters
    clst_pt = others.location.loc[others.longlat.apply(lambda x: x == clst_pt)].reset_index(drop = True)

    return pd.Series([clst_pt.values[0], distnce])


# fractal nesting and agb data
fpn = pd.read_csv("../Data/Fractal_point_nesting.csv")
agb = pd.read_csv("../Data/AGB.csv")

fpn["FirstOrder"] = fpn.FirstOrder.str.partition("_")[2].astype(int)
fpn.columns = ["site",
               "habitat",
               "logging",
               "frag_area",
               "first_order",
               "second_order",
               "third_order",
               "fourth_order",
               "fifth_order"]
fractals = fpn.loc[:, ["first_order", "second_order"]]

# specific wanted columns - and rename ***(going with Chave moist)***
agb = agb[["field_name", "Plot", "Date", "AGB_Chave_moist", "ForestQuality"]]
agb.columns = ["field_name", "second_order", "date", "agb", "forestquality"]

## Mammals Data

#### Open and sort the data

In [4]:
# open each seperate plot
E  = pd.read_csv("../Data/small_mammals/test/E_test.csv")
F  = pd.read_csv("../Data/small_mammals/test/F_test.csv")
D  = pd.read_csv("../Data/small_mammals/test/D_test.csv")
#OG = pd.read_csv("../Data/small_mammals/test/OG_test.csv")

#### function to sort out each plot in turn

In [5]:
def sort_mams(df):

    # new column names
    ncolnames   = ["occasion", "date", "grid", "point", "trap", "trap_id", "species"]
    df.columns  = ncolnames

    # stupid formatting sorted
    df["occasion"]  = df.occasion.str.replace("--", "-")
    df["plot"]      = df.occasion.str[0]
    #df["plot"]      = df.grid  **think it would be worth giving this another go!**
    df["grid"]      = df.grid.str.replace("--", "-")
    df["trap_id"]   = df.trap_id.str.replace("--", "-")
    df["trap_id"]   = df.trap_id.apply(lambda x: x[:-1])
    df["date"]      = pd.to_datetime(df.date)
    df["year"]      = df.date.dt.year
    df["census"]    = df.occasion.str.partition("-")[2].str.partition("-")[2]
    
    return df

E  = sort_mams(E)
F  = sort_mams(F)
D  = sort_mams(D)
#OG = sort_mams(OG)

frames = [E, F, D]

mamls_df = pd.concat(frames, sort = False)

mamls_df["species"] = mamls_df.species.fillna("None")

#### match with species names from lookup

In [6]:
# mammals species lookup table 
m_lkup = pd.read_csv("../Data/small_mammals/mammals_lookup.csv")
m_lkup.columns = ["code", "species", "scientific"]

mamls_df["species"] = mamls_df.species.str.strip()


# my fairly questionable decisions...

# if its a questionmark - I just go with it
# if its an either or I go with the first one!
mamls_df.loc[mamls_df.loc[:, "species"] == "CTRS-but see notes", "species"] = "CTRS"

mamls_df.loc[mamls_df.loc[:, "species"] == "SS?",          "species"] = "SS"
mamls_df.loc[mamls_df.loc[:, "species"] == "WH?",          "species"] = "WH"
mamls_df.loc[mamls_df.loc[:, "species"] == "PR?",          "species"] = "PR"
mamls_df.loc[mamls_df.loc[:, "species"] == "RR?",          "species"] = "RR"
mamls_df.loc[mamls_df.loc[:, "species"] == "MR?",          "species"] = "MR"
mamls_df.loc[mamls_df.loc[:, "species"] == "MR??",         "species"] = "MR"
mamls_df.loc[mamls_df.loc[:, "species"] == "RS?",          "species"] = "RS"
mamls_df.loc[mamls_df.loc[:, "species"] == "LGTRS?",       "species"] = "LGTRS"
mamls_df.loc[mamls_df.loc[:, "species"] == "BS?",          "species"] = "BS"
mamls_df.loc[mamls_df.loc[:, "species"] == "PSQ",          "species"] = "LSQ"      # not confident on this
mamls_df.loc[mamls_df.loc[:, "species"] == "BSQ",          "species"] = "BSQ?"
mamls_df.loc[mamls_df.loc[:, "species"] == "SSQ",          "species"] = "SSQ?"
mamls_df.loc[mamls_df.loc[:, "species"] == "RS or SS" ,    "species"] = "RS"
mamls_df.loc[mamls_df.loc[:, "species"] == "WH or SS",     "species"] = "WH"
mamls_df.loc[mamls_df.loc[:, "species"] == "BS/RS?",       "species"] = "BS"
mamls_df.loc[mamls_df.loc[:, "species"] == "PTSQ?",        "species"] = "PTSQ"
mamls_df.loc[mamls_df.loc[:, "species"] == "LETRS",        "species"] = "LETRS?"   # for some reason the lookup table has a ?
mamls_df.loc[mamls_df.loc[:, "species"] == "CBS?",         "species"] = "CBS"
mamls_df.loc[mamls_df.loc[:, "species"] == "SL?TRS",       "species"] = "SLTRS"
mamls_df.loc[mamls_df.loc[:, "species"] == "SLTRS?",       "species"] = "SLTRS"
mamls_df.loc[mamls_df.loc[:, "species"] == "L?TRS",        "species"] = "SLTRS"
mamls_df.loc[mamls_df.loc[:, "species"] == "LSQ?",         "species"] = "LSQ"
mamls_df.loc[mamls_df.loc[:, "species"] == "CTRS?",        "species"] = "CTRS"
mamls_df.loc[mamls_df.loc[:, "species"] == "LTRS or CTRS", "species"] = "CTRS"     # went with CTRS as LTRS could refer to a couple
mamls_df.loc[mamls_df.loc[:, "species"] == "LTRS",         "species"] = "LETRS?"   # not convinced about this one
mamls_df.loc[mamls_df.loc[:, "species"] == "Squirrel",     "species"] = "squirrel"
mamls_df.loc[mamls_df.loc[:, "species"] == "DTT_DEAD",     "species"] = "DTT"
mamls_df.loc[mamls_df.loc[:, "species"] == "LSQ?_DEAD",    "species"] = "LSQ"
mamls_df.loc[mamls_df.loc[:, "species"] == "squirrel",     "species"] = "unknown"  # ***mmm?***
mamls_df.loc[mamls_df.loc[:, "species"] == "See notes",    "species"] = "unknown"  # i'm effectivley treating 'unknown' as a seperate species which seems spurious at best 
mamls_df.loc[mamls_df.loc[:, "species"] == "??",           "species"] = "unknown"
mamls_df.loc[mamls_df.loc[:, "species"] == "?",            "species"] = "unknown"
mamls_df.loc[mamls_df.loc[:, "species"] == "Unknown",      "species"] = "unknown"

# merge
mamls_df = pd.merge(mamls_df,
                    m_lkup[["code", "scientific"]],
                    how      = "left",
                    left_on  = "species",
                    right_on = "code")

# get rid of the leftovers... (there were a couple of birds/reptiles)
mamls_df = mamls_df.loc[-mamls_df.code.isna(), :]

#### find closest f2 point to each trap and get agb measure

In [7]:
# get all the unique trap names
trap_locs = pd.DataFrame({"trap_id" : mamls_df.trap_id.unique()})

trap_locs = trap_locs.merge(properties[["location", "longlat"]],
                            how      = "left",
                            left_on  = "trap_id",
                            right_on = "location")

# find the closest second order fractal point
trap_locs[["second_order", "distance_so"]] = trap_locs.longlat.apply(lambda x: closest(x, second_order))

# merge back to master dataframe
mamls_df = mamls_df.merge(trap_locs, how = "left", on = "trap_id")

# just want the point number 
mamls_df.second_order = mamls_df.second_order.str[-3:]
mamls_df.second_order = mamls_df.second_order.astype(int)

# merge to get agb and forest quality
mamls_df = mamls_df.merge(agb[["second_order", "agb", "forestquality"]], how = "left",
                          on = "second_order")

#### final cleanup and save

In [8]:
mamls_df = mamls_df.rename(index=str, columns={"plot_x": "plot"})

mamls_df = mamls_df[["occasion",
                     "date",
                     "grid",
                     "point",
                     "trap",
                     "trap_id",
                     "species",
                     "year",
                     "plot",
                     "census",
                     "scientific",
                     "longlat",
                     "second_order",
                     "distance_so",
                     "agb",
                     "forestquality"]]

mamls_df.to_csv("../Results/mammals_sorted.csv")

#### Make the species/plot matrix

In [9]:
# i'll give four different combinations a go...
mamls_df["trap_year"]   = mamls_df["plot"] + "_" + mamls_df.trap_id + "_" + mamls_df.year.astype(str)
mamls_df["grid_year"]   = mamls_df["plot"] + "_" + mamls_df.grid    + "_" + mamls_df.year.astype(str)
mamls_df["trap_census"] = mamls_df["plot"] + "_" + mamls_df.trap_id + "_" + mamls_df.census
mamls_df["grid_census"] = mamls_df["plot"] + "_" + mamls_df.grid    + "_" + mamls_df.census

# function to make matrix
def make_matrix(df, what):
    mx = df.groupby([what, "species"]).size().unstack()  # groupby whatever i've chosen and species
    mx = mx.fillna(value = 0)                            # fill with 0's
    mx = mx.drop("None", axis = 1)                       # so we keep plot row even if nothing was trapped
    return mx

# do
mamls_TY = make_matrix(mamls_df, "trap_year")
mamls_GY = make_matrix(mamls_df, "grid_year")
mamls_TC = make_matrix(mamls_df, "trap_census")
mamls_GC = make_matrix(mamls_df, "grid_census")

# save
mamls_TY.to_csv("../Results/m_trap-year.csv")
mamls_GY.to_csv("../Results/m_grid-year.csv")
mamls_TC.to_csv("../Results/m_trap-census.csv")
mamls_GC.to_csv("../Results/m_grid_census.csv")

#### agb

In [10]:
# get the agb (either mean or median of all the traps in plot)
mamls_agb = mamls_df.groupby("plot").agb.describe()
mamls_agb = pd.DataFrame(mamls_agb["50%"])
mamls_agb.to_csv("../Results/mamls_agb.csv")

#### standardise time

In [11]:
# and time difference between cesuses - i'm doing year or occasion...
mamls_cn_diff = census_diff(mamls_df, "census")
mamls_yr_diff = census_diff(mamls_df, "year")

mamls_yr_diff = mamls_yr_diff.rename(index=str, columns={"year": "census"})

mamls_cn_diff.to_csv("../Results/mamls_census_dates.csv")
mamls_yr_diff.to_csv("../Results/mamls_years_dates.csv")

## Tree Data

#### readin raw data

In [12]:
# readin RAW data
trees_df = pd.read_csv("../Data/SAFE_CarbonPlots_Tree+LianaCensus.csv")

#### function to sort everything out for each census

In [13]:
def sort_data(df, census_no):  # give new column names, delete NAs and dead...

    # consistant and better column names
    new_Cnames = ['f_type',       # forest type
                  'plot',
                  'subplot',
                  'date',         # date of measurements
                  'observers',
                  'tag_no',
                  'd_pom',        # diameter of tree (cm)
                  'h_pom',        # height diameter is taken (m) 1.3 by default
                  'height',
                  'flag',         # condition of trees (see flag list)
                  'alive',        # 1 = yes, NaN = no
                  'stem_C',       # aboveground biomass of tree (kg)
                  'root_C',       # root biomass of tree
                  'field_cmnts',  # comments from field
                  'data_cmnts',   # comments from data entry
                  'sbplt_X',
                  'sbplt_Y',
                  'CPA',          # projected area of the crown of the stem
                  'X_FMC',        # plot level X coordinate
                  'Y_FMC',        # plot level Y coordinte
                  'Z_FMC',        # plot level elevation
                  'family',
                  'binomial',
                  'wood_density']

    # give each census these column names
    df.columns = new_Cnames

    # get unique ID - combine plot and tag_no
    df = df.assign(ID = df['plot'] + df['tag_no'].map(str))

    # column with census number
    df = df.assign(census = census_no)

    # delete rows with NaNs in important columns
    impt_cols = ['tag_no', 'd_pom', 'h_pom', 'height', 'flag', 'alive',
                 'stem_C', 'root_C']

    df = df.dropna(subset = impt_cols, how = 'all')

    # delete dead trees (alive == 0)
    df = df[df.alive == 1]

    # sort out dates
    df.date = pd.to_datetime(df.date, dayfirst = True)

    return df

#### subset each census - weird column things...

In [14]:
# subset for each census
census_1 = trees_df.iloc[ :, list(range(0, 3))     # same for all
                           + list(range(3, 15))    # specific for census
                           + list(range(53, 62))]  # same for all

census_2 = trees_df.iloc[ :, list(range(0, 3))
                           + list(range(15, 27))
                           + list(range(53, 62))]

census_3 = trees_df.iloc[ :, list(range(0, 3))
                           + list(range(27, 39))
                           + list(range(53, 62))]

census_4 = trees_df.iloc[ :, list(range(0, 3))
                           + list(range(39, 51))
                           + list(range(53, 62))]

#### do function and combine census'

In [15]:
# sort data for each census
census_1 = sort_data(census_1, 1)
census_2 = sort_data(census_2, 2)
census_3 = sort_data(census_3, 3)
census_4 = sort_data(census_4, 4)

# recombine all census data (stack on top of each other)
trees_df = pd.concat([census_1, census_2, census_3, census_4], ignore_index = True)

#### add extra columns and save

In [16]:
# add genus column
trees_df['plot']        = trees_df['plot'].replace(" ", "", regex=True)
trees_df['subplot']     = trees_df['subplot'].apply(lambda x: str(x).zfill(2))
trees_df['genus']       = trees_df.apply(lambda row: row.binomial.split(" ")[0], axis = 1)
trees_df['plt_sub']     = trees_df['plot'] + "_sp" + trees_df['subplot'].astype(str)
trees_df['plt_sub_cen'] = trees_df['plt_sub'] + "_c" + trees_df['census'].astype(str)
trees_df['plot_c']      = trees_df['plot'] + "_c" + trees_df['census'].astype(str)
trees_df['census']      = "c" + trees_df.census.astype(str)

# save to csv
trees_df.to_csv("../Results/trees_sorted.csv", index = False)

#### make matrix

In [17]:
# species matrix
trees_matrix = trees_df.groupby(['plt_sub_cen', 'binomial']).size().unstack()
trees_matrix = trees_matrix.fillna(value = 0)
trees_matrix.to_csv("../Results/trees_matrix.csv")

trees_genus_matrix = trees_df.groupby(['plt_sub_cen', 'genus']).size().unstack()
trees_genus_matrix = trees_genus_matrix.fillna(value = 0)
trees_genus_matrix.to_csv("../Results/trees_genus_matrix.csv")

trees_family_matrix = trees_df.groupby(['plt_sub_cen', 'family']).size().unstack()
trees_family_matrix = trees_family_matrix.fillna(value = 0)
trees_family_matrix.to_csv("../Results/trees_family_matrix.csv")

#### trees agb (different from the others as calculated from data)

In [18]:
# total biomass at each census for each plot
trees_agb = pd.DataFrame(trees_df.groupby(["plot", "census"]).stem_C.sum())

# take the mean of all census' 0.0625 and 0.001 to get it into Mg/0.0625 ha
trees_agb = pd.DataFrame(trees_agb.groupby("plot").mean()*  0.0625 * 0.001)

trees_agb.to_csv("../Results/trees_agb.csv")

#### trees census standardise time

In [19]:
trees_cen = census_diff(trees_df, "census")
trees_cen.to_csv("../Results/trees_census_dates.csv")

## Beetles Data

#### read in raw data

In [20]:
btles_df = pd.read_csv("../Data/family_list.csv", index_col = 0)

# convert dates to datetime
btles_df.date = pd.to_datetime(btles_df.date, dayfirst = True)

#### initial sorting

In [21]:
btles_df = btles_df.merge(fractals, how = "left", left_on = "trap_N", right_on = "first_order")
btles_df["second_order"] = btles_df["second_order"].str.partition("_")[2].astype(int)
btles_df = btles_df.merge(agb[["second_order", "agb", "forestquality"]], how = "left",
                          on = "second_order")
btles_df = btles_df.rename(index=str, columns={"block": "plot"})

btles_df["subplot"] = btles_df["plot"] + "-" + btles_df.first_order.astype(str)

#### sorting out census'

In [22]:
# sampling periods
btles_df['census'] = "incomplete"


def sample_period(df, s_date, e_date, period):
    df.loc[(df.date >= s_date) & (df.date < e_date), 'census'] = period

s1 = pd.to_datetime("01/01/2011", dayfirst = True)  # might be good to check!!
e1 = pd.to_datetime("01/04/2011", dayfirst = True)
s2 = pd.to_datetime("01/09/2011", dayfirst = True)
e2 = pd.to_datetime("01/01/2012", dayfirst = True)
s3 = pd.to_datetime("01/04/2012", dayfirst = True)
e3 = pd.to_datetime("01/09/2012", dayfirst = True)

sample_period(btles_df, s1, e1, "P1")
sample_period(btles_df, s2, e2, "P2")
sample_period(btles_df, s3, e3, "P3")

btles_df = btles_df[btles_df.census != "incomplete"]

btles_df['plt_sub_cen'] = btles_df["plot"] + "_" + btles_df.subplot + "_" + btles_df.census

btles_df.to_csv("../Results/btles_sorted.csv", index = False)


btles_mx = btles_df.groupby(['plt_sub_cen', 'family']).size().unstack()
btles_mx = btles_mx.fillna(value = 0)
btles_mx.to_csv("../Results/btles_matrix.csv")

#### time between censuses

In [23]:
btles_cen = census_diff(btles_df, "census")
btles_cen.to_csv("../Results/btles_census_dates.csv")

#### agb

In [24]:
btles_agb = pd.DataFrame(btles_df.groupby("plot").agb.median())

btles_agb.to_csv("../Results/btles_agb.csv")

## Mozzies!

In [70]:
mozie1 = pd.read_csv("../Data/DailyHLC2012-2013.csv")
mozie2 = pd.read_csv("../Data/DailyHLC2013-2014.csv")

In [71]:
mozie1["census"] = "c1"
mozie2["census"] = "c2"

In [72]:
frames = [mozie1, mozie2]
mozie  = pd.concat(frames, sort = False)

In [73]:
mozie = mozie.drop(['field_name',
                    'Collector',
                    'Moonlight',
                    'Forest_cover',
                    'Height',
                    'Tree_Height',
                    'Wind',
                    'Rain',
                    'Temperature',
                    'Humidity'], axis = 1)

In [74]:
mozie_meta  = mozie[['Date', 'Location', 'Disturbance', 'census']]
mozie_count = mozie.drop(['Date', 'Location', 'Disturbance', 'census'], axis = 1)

mozie_meta.columns = ["date", "location", "disturbance", "census"]

mozie_count = mozie_count.fillna(0)
mozie_count.columns = pd.Series(mozie_count.columns).str.rpartition("_")[0]

In [75]:
frames = [mozie_meta, mozie_count]
mozie = pd.concat([mozie_meta, mozie_count], axis = 1,sort = False)
mozie = mozie.reset_index()

mozie = mozie.drop("index", axis = 1)

In [76]:
mozie["date"] = pd.to_datetime(mozie.date)

In [77]:
# get rid of Da_Tree
mozie = mozie.loc[mozie.location.str.rpartition("_")[0] != "Da_Tree", :]

In [78]:
mozie[["plot", "second_order"]] = mozie.location.str.partition("_")[[0,2]]

mozie.second_order = mozie.second_order.astype(int)

mozie = mozie.merge(agb[["second_order", "agb", "forestquality"]], how = "left", on = "second_order")

In [79]:
mozie['plt_sub_cen'] = mozie["plot"] + "_" + mozie.second_order.astype(str) + "_" + mozie.census

In [80]:
mozie.to_csv("../Results/mozie_sorted.csv")

In [81]:
mozie.index = mozie.plt_sub_cen

mozie_mx = mozie.drop(["date",
                       "location",
                       "disturbance",
                       "census",
                       "agb",
                       "forestquality",
                       "plot",
                       "second_order",
                       "plt_sub_cen"], axis = 1)

In [83]:
mozie_cen = census_diff(mozie, "census")
mozie_cen.to_csv("../Results/mozie_census_dates.csv")

In [84]:
mozie_agb = pd.DataFrame(mozie.groupby("plot").agb.median())

mozie_agb.to_csv("../Results/btles_agb.csv")

In [85]:
mozie_agb

,agb
plot,
D,1.217018
E,4.140394
OG2,29.274437
OG3,18.551408
OP1,0.186782
OP2,0.321447
OP3,0.000000
VJR,15.262123
